#Data Generation - News Articles Corpus

##Scroll and History News Network articles

In [0]:
#from multiprocessing import Pool

In [0]:
import time
import requests
from bs4 import BeautifulSoup as bs
import csv
import pandas as pd
import re,datetime
start_time = time.time()

class Corpus:
    def __init__(self,web_url,start,end,filename,fieldnames):
      self.web_url=web_url
      self.start=start
      self.end=end
      self.rawdata=[]
      self.article_id=[]
      self.headline = []
      self.dates = []
      self.category = []
      self.author = []
      self.articles = []
      self.filename=filename
      self.fieldnames=fieldnames

    def gen_corpus(self):
      for i in range(self.start,self.end): #article numbers
          self.article_id.append(str(int(i)))
          rd=requests.get(self.web_url+'article/'+str(int(i))).text
          self.rawdata.append(rd)
      print(len(self.rawdata))
      print("%s seconds" % (time.time() - start_time))

    def gen_corpus_attr(self):

      for i in range (len(self.rawdata)):
          # Turn page into BeautifulSoup object to access HTML tags
          soup = bs(self.rawdata[i])

          #to get headline
          try:
              head = soup.find('h1').get_text()
          except:
              head = None
          #to get date  (the date in the webpeg was in the info class with some other info, so it needed to be extracted)
          try:
              date = soup.find_all(class_="article-info")
              datetext = [tag.get_text().strip() for tag in date]
              date = datetext[-1]
          except:
              date = None
          #to get category
          try:
              cat = soup.find(style="color:#777;").get_text().strip()
          except:
              cat = None
          #to get other
          try:
              auth = soup.find(class_="byline").get_text()
          except:
              auth = None

          #to extract text
          p_tags = soup.find_all('p')
          # Get the text from each of the "p" tags and strip surrounding whitespace.
          p_tags_text = [tag.get_text().replace('\xa0', ' ').strip('\t') for tag in p_tags] #to remove non-breaking spaces from the document
          article = ' '.join(p_tags_text)
          self.headline.append(str(head))
          self.dates.append(str(date))
          self.category.append(str(cat))
          self.author.append(str(auth))
          self.articles.append(str(article))

    def gen_corpusfile(self):
      f = open(self.filename, "w")
      writer = csv.DictWriter(f, fieldnames=self.fieldnames)
      writer.writeheader()
      f.close()
      with open(self.filename, 'a') as f:
          writer = csv.writer(f)
          writer.writerows(zip(self.article_id, self.dates, self.author, self.category, self.headline, self.articles))

c1_scroll=Corpus('https://scroll.in/',896000,904500,'scrollnews.csv',['Article_id', 'Date', 'Author', 'Topic', 'Headline', 'Article content'])
c2_hnn=Corpus('https://historynewsnetwork.org/',170100,172101,'HNN.csv',['Article_id', 'Date', 'Author', 'Topic', 'Headline','Article content'])

c2_hnn.gen_corpus()
c2_hnn.gen_corpus_attr()
c2_hnn.gen_corpusfile()
#896000 to 904500
#170100 to 172101

2001
244.63873147964478 seconds


In [0]:
c1_scroll.gen_corpus()
c1_scroll.gen_corpus_attr()
c1_scroll.gen_corpusfile()

8500
18478.092730998993 seconds


In [0]:
dfhnn=pd.read_csv(c2_hnn.filename)
dfhnn.head()

,Article_id,Date,Author,Topic,Headline,Article content
1996,172096,5/27/2019,by Jim Sleeper,Roundup,Billionaires can't fix college: Jim Sleeper on...,Jim Sleeper is a lecturer in political scienc...
1997,172097,5/8/2019,by Ralph Seliger,Roundup,My Zionism is Personal and Complicated,Ralph Seliger specializes in writing about Is...
1998,172098,6/2/19,by Donald J. Fraser,News at Home,A Brief History of the Theory Trump and Barr U...,Donald J. Fraser has spent a lifetime working...
1999,172099,6/2/19,by Gregory Sumner,News Abroad,Remembering Rome's Liberation,Gregory Sumner is author of Detroit in WWII a...
2000,172100,6/2/19,by William Lambers,News at Home,D-Day 75 Years Later and the Quest for Peace,William Lambers is the author of the Road to ...


In [0]:
dfscroll=pd.read_csv(c1_scroll.filename)


,Article_id,Date,Author,Topic,Headline,Article content
8495,904495,None,None,None,Government is using litigation to intimidate C...,Former Information Commissioner Sridhar Achar...
8496,904496,None,None,None,Telangana polls: Congress urges EC to act agai...,The Congress on Tuesday urged the Election Co...
8497,904497,None,None,None,Jose Mourinho denies stating it would be a mir...,Jose Mourinho has acknowledged Manchester Uni...
8498,904498,None,None,None,Tottenham fan escapes racism charge after bana...,The Tottenham Hotspur fan who threw a banana ...
8499,904499,None,None,None,Sri Lanka: Rajapaksa moves SC against order re...,Sri Lankan Prime Minister Mahinda Rajapaksa o...


In [0]:
dfscroll.isnull().sum()

##News Articles from The Verge, ESPN and Sports Bible



In [0]:
!pip install newspaper3k
import newspaper
import time
import re
import pandas as pd

def fetch_newspaper(url):
    source = newspaper.build(url, memoize_articles=False)
    return source

def see_urls(source):
    print('{} articles fetched'.format(source.size()))
    for article in source.articles:
        print(article.url)

def batch_download_articles(source):
    start_time = time.time()
    newspaper.news_pool.set([source])
    newspaper.news_pool.join()
    print('Downloaded {} articles in {} seconds'.format(source.size(),time.time() - start_time))

def parse_articles(source):
    start_time = time.time()
    for article in source.articles:
          article.parse()
    print('Parsed {} articles in {} seconds'.format(source.size(),time.time() - start_time))
          
def clean(article):
    article = re.sub('\s+',' ',article)
    return article
          
def create_DataFrame(source, category):
    df = pd.DataFrame(data={"Article_id":[None for i in range(source.size())]
                                 ,"Date": [str(article.publish_date) for article in source.articles]
                                 ,"Author": [article.authors for article in source.articles]
                                 ,"Topic": [category for i in range(source.size())]
                                 ,"Headline": [article.title for article in source.articles]
                                 ,"Article content": [clean(article.text) for article in source.articles]
                                 })
    return df

In [0]:
#tech articles from The Verge
verge = fetch_newspaper('https://www.theverge.com/')
#look for any unnecessary articles
see_urls(verge)

#remove unnecessary articles
comment_urls = [article.url for article in verge.articles if 'comments' in article.url.split("#")]
for url in comment_urls:
    for article in verge.articles:
        if url == article.url:
            verge.articles.remove(article)

#see the changes
see_urls(verge)

#download and parse all the articles to extract all the relevant info
batch_download_articles(verge)
parse_articles(verge)

#create a pandas dataframe and save as .csv file
verge_df = create_DataFrame(verge, 'technology')
verge_df.to_csv('verge_articles.csv')

#-----------------------------------------------------------------------
#sports articles from ESPN
espn = fetch_newspaper('https://www.espn.in/')
#look for any unnecessary articles
see_urls(espn)

#remove unnecessary articles
videos = [article.url for article in espn.articles if 'video' in article.url.split("/")]
for url in videos:
    for article in espn.articles:
        if url == article.url:
            espn.articles.remove(article)

#see the changes
see_urls(espn)

#download and parse all the articles to extract all the relevant info
batch_download_articles(espn)
parse_articles(espn)

#create a pandas dataframe and save as .csv file
espn_df = create_DataFrame(espn, 'sports')
espn_df.to_csv('espn_articles.csv')

#----------------------------------------------------------------------------
#more sports articles from Sport Bible
sport_bible = fetch_newspaper('https://www.sportbible.com/')
#look for any unnecessary articles
# see_urls(sport_bible)
print('{} Articles available'.format(sport_bible.size()))

#remove unnecessary articles
unnecessary = [article.url for article in sport_bible.articles if 'pages' in article.url.split("/")]
for url in unnecessary:
    for article in sport_bible.articles:
        if url == article.url:
            sport_bible.articles.remove(article)
            
#see the changes
# see_urls(sport_bible)
print('{} Articles after deletion'.format(sport_bible.size()))

#download and parse all the articles to extract all the relevant info
print('Downloading articles...')
batch_download_articles(sport_bible)
print('Parsing Articles')
parse_articles(sport_bible)

#create a pandas dataframe and save as .csv file
sport_bible_df = create_DataFrame(sport_bible, 'sports')
print('.csv file created')
sport_bible_df.to_csv('sport_bible_articles.csv')

In [0]:
verge = pd.read_csv('verge_articles.csv')
espn = pd.read_csv('espn_articles.csv')
sport_bible = pd.read_csv('sport_bible_articles.csv')

df = verge.copy()
df = df.append([espn,sport_bible])
df = df.drop(columns=['Unnamed: 0'])

df = df.reset_index()
df = df.drop(columns=['index'])

print(df)
df.to_csv('compiled_news.csv')

      Article_id  ...                                    Article content
0            NaN  ...  Twitter is testing a new feature that lets use...
1            NaN  ...  Only the best deals on Verge-approved gadgets ...
2            NaN  ...  President Trump wants to create a panel that w...
3            NaN  ...  I forgot to write about Westworld after its th...
4            NaN  ...  They’re pulling the plug on the New York Auto ...
...          ...  ...                                                ...
2144         NaN  ...  Even though the World Cup is over and we're le...
2145         NaN  ...  Social media was ablaze with comments about Ki...
2146         NaN  ...  Ferrari driver Kimi Raikkonen is getting a lot...
2147         NaN  ...  Jack Kenmare Jack Kenmare is a writer at SPORT...
2148         NaN  ...  The manager of former F1 champion Michael Schu...

[2149 rows x 6 columns]


In [0]:
len(df)

2149

##Final corpus - combine news from all sources

In [0]:
#to remove missing values from the final news corpus

import pandas as pd
missing_values = ["n/a", "na", "--","None","nan","nil","NaN","Null","none"]
scrollarticles1 = pd.read_csv(c1_scroll.filename, na_values = missing_values)
del scrollarticles1['Unnamed: 0']


In [0]:
HNNarticles = pd.read_csv(c2_hnn.filename, na_values = missing_values)
del HNNarticles1['Unnamed: 0']

In [0]:
compiledarticles = pd.read_csv('compiled_news.csv', na_values = missing_values)
del compiledarticles['Unnamed: 0']

In [0]:
import pandas as pd
frames = [scrollarticles, scrollarticles1, HNNarticles1, compiledarticles1]
corpus = pd.concat(frames)
corpus = corpus[corpus['Article content'].notna()]
print(len(corpus))
corpus.columns.values

In [0]:
corpus.to_csv("corpus_final.csv")

In [0]:
data=pd.read_csv('corpus_final.csv', index_col=[0])
print(len(data))
data.head()